# GNN Training on 10 Users' Trajectories

This notebook:
- Loads 10 specific users' trajectories: ['000', '001', '005', '006', '009', '011', '014', '016', '019', '025']
- Removes consecutive duplicates (AAABCDCCABB → ABCDCAB) for each user
- Builds graph structure (nodes = places, edges = transitions)
- Trains GNN model (GCN + LSTM) for location prediction
- Evaluates all 4 metrics: Accuracy, Precision & Recall, Top-K Accuracy, MPD


## Section 1 — Imports & Setup


In [79]:
import os
import pandas as pd
import numpy as np
import json
import pickle
from tqdm import tqdm
from math import radians, sin, cos, sqrt, atan2
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import precision_score, recall_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch_geometric.data import Data
from torch_geometric.nn import GCNConv, SAGEConv, GATConv
import warnings
warnings.filterwarnings('ignore')

# Manual haversine implementation to avoid dependency issues
def haversine(coord1, coord2):
    """
    Calculate the great circle distance between two points on Earth.
    
    Parameters:
    coord1: tuple of (latitude, longitude) in degrees
    coord2: tuple of (latitude, longitude) in degrees
    
    Returns:
    Distance in kilometers
    """
    lat1, lon1 = coord1
    lat2, lon2 = coord2
    
    # Convert to radians
    lat1, lon1, lat2, lon2 = map(radians, [lat1, lon1, lat2, lon2])
    
    # Haversine formula
    dlat = lat2 - lat1
    dlon = lon2 - lon1
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * atan2(sqrt(a), sqrt(1-a))
    
    # Earth radius in kilometers
    R = 6371.0
    
    return R * c

# Set random seeds
np.random.seed(42)
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed(42)

# Paths
BASE_PATH = "/home/root495/Inexture/Location Prediction Update"
PROCESSED_PATH = BASE_PATH + "/data/processed/"
SEQUENCES_FILE = PROCESSED_PATH + "place_sequences.json"
GRAPH_EDGES_FILE = PROCESSED_PATH + "graph_edges.csv"
NODE_FEATURES_FILE = PROCESSED_PATH + "node_features.csv"
GRID_METADATA_FILE = PROCESSED_PATH + "grid_metadata.json"
CLEANED_WITH_PLACES_FILE = PROCESSED_PATH + "cleaned_with_places.csv"
OUTPUT_PATH = BASE_PATH + "/notebooks/"
MODELS_PATH = BASE_PATH + "/models/"
RESULTS_PATH = BASE_PATH + "/results/"
MODEL_SAVE_PATH = MODELS_PATH + "gnn_10users_model_best.pt"
RESULTS_SAVE_PATH = RESULTS_PATH + "gnn_10users_results.json"

os.makedirs(OUTPUT_PATH, exist_ok=True)
os.makedirs(MODELS_PATH, exist_ok=True)
os.makedirs(RESULTS_PATH, exist_ok=True)

# Specific users to use
SELECTED_USERS = ['000', '001', '005', '006', '009', '011', '014', '016', '019', '025']

# Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Optimize for speed
if torch.cuda.is_available():
    torch.backends.cudnn.benchmark = True  # Faster convolutions
    torch.backends.cudnn.deterministic = False  # Allow non-deterministic for speed

print("Libraries imported successfully!")


Using device: cpu
Libraries imported successfully!


## Section 2 — Load 10 Users' Trajectories


In [80]:
# Load place sequences
print("Loading place sequences...")
with open(SEQUENCES_FILE, 'r') as f:
    sequences_dict = json.load(f)

print(f"Total users available: {len(sequences_dict)}")

# Load sequences for specific users
user_sequences = {}
total_places = 0
for user_id in SELECTED_USERS:
    if user_id in sequences_dict:
        seq = sequences_dict[user_id]
        user_sequences[user_id] = seq
        total_places += len(seq)
        print(f"  User {user_id}: {len(seq)} places")
    else:
        print(f"  Warning: User {user_id} not found in sequences!")

print(f"\nSelected {len(user_sequences)} users: {list(user_sequences.keys())}")
print(f"Total places across all users: {total_places}")


Loading place sequences...
Total users available: 54
  User 000: 173817 places
  User 001: 108561 places
  User 005: 108967 places
  User 006: 31809 places
  User 009: 84573 places
  User 011: 90770 places
  User 014: 388051 places
  User 016: 89208 places
  User 019: 47792 places
  User 025: 628816 places

Selected 10 users: ['000', '001', '005', '006', '009', '011', '014', '016', '019', '025']
Total places across all users: 1752364


## Section 3 — Preprocess: Remove Consecutive Duplicates

Remove consecutive duplicate locations for each user. Example: AAABCDCCABB → ABCDCAB

Only consecutive duplicates are removed. If a location appears again later (non-consecutive), it is kept.


In [81]:
def remove_consecutive_duplicates(sequence):
    """
    Remove consecutive duplicates from sequence.
    Example: [A, A, A, B, C, D, C, C, A, B, B] → [A, B, C, D, C, A, B]
    """
    if len(sequence) == 0:
        return sequence
    
    processed = [sequence[0]]  # Always keep first element
    
    for i in range(1, len(sequence)):
        # Only add if different from previous (not consecutive duplicate)
        if sequence[i] != sequence[i-1]:
            processed.append(sequence[i])
    
    return processed

# Apply consecutive duplicate removal to each user
processed_sequences = {}
total_original = 0
total_processed = 0

print("Processing users...")
for user_id in tqdm(list(user_sequences.keys()), desc="Removing duplicates"):
    original_seq = user_sequences[user_id]
    processed_seq = remove_consecutive_duplicates(original_seq)
    processed_sequences[user_id] = processed_seq
    
    original_len = len(original_seq)
    processed_len = len(processed_seq)
    total_original += original_len
    total_processed += processed_len
    
    reduction = original_len - processed_len
    reduction_pct = (reduction / original_len * 100) if original_len > 0 else 0
    print(f"  User {user_id}: {original_len} → {processed_len} places ({reduction_pct:.1f}% reduction)")

print(f"\nSummary:")
print(f"  Total original places: {total_original}")
print(f"  Total after processing: {total_processed}")
print(f"  Total duplicates removed: {total_original - total_processed} ({((total_original - total_processed)/total_original*100):.1f}%)")


Processing users...


Removing duplicates:  70%|███████   | 7/10 [00:00<00:00, 47.77it/s]

  User 000: 173817 → 795 places (99.5% reduction)
  User 001: 108561 → 186 places (99.8% reduction)
  User 005: 108967 → 283 places (99.7% reduction)
  User 006: 31809 → 103 places (99.7% reduction)
  User 009: 84573 → 17 places (100.0% reduction)
  User 011: 90770 → 125 places (99.9% reduction)
  User 014: 388051 → 766 places (99.8% reduction)
  User 016: 89208 → 124 places (99.9% reduction)
  User 019: 47792 → 120 places (99.7% reduction)


Removing duplicates: 100%|██████████| 10/10 [00:00<00:00, 42.59it/s]

  User 025: 628816 → 1568 places (99.8% reduction)

Summary:
  Total original places: 1752364
  Total after processing: 4087
  Total duplicates removed: 1748277 (99.8%)


## Section 4 — Build Graph Structure

Build graph from trajectory data:
- Nodes = unique place IDs
- Edges = transitions between places
- Edge weights = transition frequencies


In [82]:
# Build graph from sequences
print("Building graph structure...")

# Collect all unique places
all_places = set()
for seq in processed_sequences.values():
    all_places.update(seq)

# Create place to index mapping
place_to_idx = {place: idx for idx, place in enumerate(sorted(all_places))}
idx_to_place = {idx: place for place, idx in place_to_idx.items()}
num_nodes = len(place_to_idx)

print(f"Total unique places (nodes): {num_nodes}")

# Build edge list and weights from sequences
edge_index = []
edge_weights = []
transition_counts = {}

for seq in processed_sequences.values():
    for i in range(len(seq) - 1):
        source = seq[i]
        target = seq[i+1]
        
        source_idx = place_to_idx[source]
        target_idx = place_to_idx[target]
        
        # Count transitions
        if (source_idx, target_idx) not in transition_counts:
            transition_counts[(source_idx, target_idx)] = 0
        transition_counts[(source_idx, target_idx)] += 1

# Create edge list and weights
for (source_idx, target_idx), count in transition_counts.items():
    edge_index.append([source_idx, target_idx])
    edge_weights.append(count)

edge_index = torch.tensor(edge_index, dtype=torch.long).t().contiguous()
edge_weights = torch.tensor(edge_weights, dtype=torch.float)

print(f"Total edges: {len(edge_weights)}")
print(f"Edge index shape: {edge_index.shape}")
print(f"Graph built successfully!")


Building graph structure...
Total unique places (nodes): 303
Total edges: 690
Edge index shape: torch.Size([2, 690])
Graph built successfully!


## Section 5 — Create Sequences for Training

Split each user's processed sequence into fixed-length chunks of 50 events each.
Combine sequences from all users for training.


In [83]:
# Create sequences of fixed length 50
SEQUENCE_LENGTH = 50

# Use sliding windows for more training data (balanced overlap)
# Create overlapping sequences with step size of 25 (50% overlap - good for learning)
all_sequences = []
step_size = 25  # 50% overlap for better learning

print("Creating sequences from all users...")
for user_id in tqdm(list(processed_sequences.keys()), desc="Processing users"):
    processed_seq = processed_sequences[user_id]
    user_sequences_list = []
    
    for i in range(0, len(processed_seq) - SEQUENCE_LENGTH + 1, step_size):
        chunk = processed_seq[i:i+SEQUENCE_LENGTH]
        if len(chunk) == SEQUENCE_LENGTH:  # Only full-length sequences
            user_sequences_list.append(chunk)
    
    all_sequences.extend(user_sequences_list)
    print(f"  User {user_id}: {len(user_sequences_list)} sequences")

print(f"\nTotal sequences created: {len(all_sequences)}")
print(f"Total events in sequences: {sum(len(s) for s in all_sequences)}")

# Split into train/test (80/20)
split_idx = int(len(all_sequences) * 0.8)
train_sequences = all_sequences[:split_idx]
test_sequences = all_sequences[split_idx:]

print(f"\nTraining sequences: {len(train_sequences)}")
print(f"Test sequences: {len(test_sequences)}")

if len(test_sequences) == 0:
    # If no test sequences, use last training sequence for testing
    test_sequences = [train_sequences[-1]]
    train_sequences = train_sequences[:-1]
    print(f"Adjusted: Training={len(train_sequences)}, Test=1 (using last training sequence)")


Creating sequences from all users...


Processing users: 100%|██████████| 10/10 [00:00<00:00, 19991.92it/s]

  User 000: 30 sequences
  User 001: 6 sequences
  User 005: 10 sequences
  User 006: 3 sequences
  User 009: 0 sequences
  User 011: 4 sequences
  User 014: 29 sequences
  User 016: 3 sequences
  User 019: 3 sequences
  User 025: 61 sequences

Total sequences created: 149
Total events in sequences: 7450

Training sequences: 119
Test sequences: 30


## Section 6 — Prepare Node Features

Extract node features for each place:
- Visit frequency
- Coordinates (lat, lon)


In [ ]:
# Load coordinates
print("Loading coordinates...")
df_places = pd.read_csv(CLEANED_WITH_PLACES_FILE)
place_coords = df_places.groupby('place_id')[['lat', 'lon']].first().to_dict('index')

# Load grid metadata for fallback
with open(GRID_METADATA_FILE, 'r') as f:
    grid_metadata = json.load(f)

# Calculate visit frequency for each place
print("Calculating visit frequencies...")
place_visit_counts = {}
for seq in train_sequences + test_sequences:
    for place in seq:
        place_visit_counts[place] = place_visit_counts.get(place, 0) + 1

# Calculate in-degree and out-degree for each node
print("Calculating node degrees...")
in_degree = {idx: 0 for idx in range(num_nodes)}
out_degree = {idx: 0 for idx in range(num_nodes)}
transition_freq = {idx: {} for idx in range(num_nodes)}

for seq in train_sequences + test_sequences:
    indices = [place_to_idx[place] for place in seq]
    for i in range(len(indices) - 1):
        source_idx = indices[i]
        target_idx = indices[i+1]
        out_degree[source_idx] += 1
        in_degree[target_idx] += 1
        if target_idx not in transition_freq[source_idx]:
            transition_freq[source_idx][target_idx] = 0
        transition_freq[source_idx][target_idx] += 1

# Calculate transition probability (max transition prob from this node)
max_transition_prob = {}
for source_idx in range(num_nodes):
    if len(transition_freq[source_idx]) > 0:
        total = sum(transition_freq[source_idx].values())
        max_prob = max(transition_freq[source_idx].values()) / total if total > 0 else 0.0
        max_transition_prob[source_idx] = max_prob
    else:
        max_transition_prob[source_idx] = 0.0

# First, collect all coordinates to calculate min/max for normalization
print("Collecting coordinates for normalization...")
all_lats = []
all_lons = []
for idx in range(num_nodes):
    place_id = idx_to_place[idx]
    if place_id in place_coords:
        all_lats.append(place_coords[place_id]['lat'])
        all_lons.append(place_coords[place_id]['lon'])
    else:
        # Fallback: calculate from grid
        try:
            if "_" in str(place_id):
                row, col = map(int, str(place_id).split("_"))
                lat = grid_metadata['min_lat'] + row * grid_metadata['deg_lat']
                lon = grid_metadata['min_lon'] + col * grid_metadata['deg_lon']
                all_lats.append(lat)
                all_lons.append(lon)
        except:
            pass

# Calculate min/max for normalization
min_lat = min(all_lats) if all_lats else grid_metadata['min_lat']
max_lat = max(all_lats) if all_lats else grid_metadata['min_lat'] + 100 * grid_metadata['deg_lat']
min_lon = min(all_lons) if all_lons else grid_metadata['min_lon']
max_lon = max(all_lons) if all_lons else grid_metadata['min_lon'] + 100 * grid_metadata['deg_lon']

print(f"Coordinate ranges: lat=[{min_lat:.4f}, {max_lat:.4f}], lon=[{min_lon:.4f}, {max_lon:.4f}]")

# Normalize degrees
max_in_degree = max(in_degree.values()) if in_degree.values() else 1
max_out_degree = max(out_degree.values()) if out_degree.values() else 1

# Build enhanced node features
node_features = []
for idx in range(num_nodes):
    place_id = idx_to_place[idx]
    
    # Feature 1: Visit frequency (normalized)
    visit_freq = place_visit_counts.get(place_id, 0)
    max_visits = max(place_visit_counts.values()) if place_visit_counts else 1
    visit_freq_norm = visit_freq / max_visits if max_visits > 0 else 0.0
    
    # Feature 2 & 3: Coordinates
    if place_id in place_coords:
        lat = place_coords[place_id]['lat']
        lon = place_coords[place_id]['lon']
    else:
        # Fallback: calculate from grid
        try:
            if "_" in str(place_id):
                row, col = map(int, str(place_id).split("_"))
                lat = grid_metadata['min_lat'] + row * grid_metadata['deg_lat']
                lon = grid_metadata['min_lon'] + col * grid_metadata['deg_lon']
            else:
                lat, lon = min_lat, min_lon  # Use min as default
        except:
            lat, lon = min_lat, min_lon
    
    # Normalize coordinates
    lat_norm = (lat - min_lat) / (max_lat - min_lat + 1e-8)
    lon_norm = (lon - min_lon) / (max_lon - min_lon + 1e-8)
    
    # Feature 4: In-degree (normalized)
    in_deg_norm = in_degree[idx] / max_in_degree if max_in_degree > 0 else 0.0
    
    # Feature 5: Out-degree (normalized)
    out_deg_norm = out_degree[idx] / max_out_degree if max_out_degree > 0 else 0.0
    
    # Feature 6: Max transition probability
    max_trans_prob = max_transition_prob[idx]
    
    node_features.append([visit_freq_norm, lat_norm, lon_norm, in_deg_norm, out_deg_norm, max_trans_prob])

node_features = torch.tensor(node_features, dtype=torch.float)
print(f"Node features shape: {node_features.shape}")
print(f"Node features: [visit_frequency, lat, lon, in_degree, out_degree, max_transition_prob]")


Loading coordinates...
Calculating visit frequencies...
Calculating node degrees...
Coordinate ranges: lat=[18.2817, 41.1173], lon=[109.1517, 121.8167]
Node features shape: torch.Size([303, 6])
Node features: [visit_frequency, lat, lon, in_degree, out_degree, max_transition_prob]


## Section 7 — Train GNN Model

Define and train GCN + LSTM model for sequence prediction.


In [85]:
# Define enhanced GNN + LSTM model (optimized for best performance)
class ImprovedGNNLSTM(nn.Module):
    def __init__(self, num_nodes, node_feature_dim, hidden_dim=224, gnn_layers=4, lstm_layers=2, dropout=0.2):
        super(ImprovedGNNLSTM, self).__init__()
        
        self.num_nodes = num_nodes
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        
        # GraphSAGE layers (better embeddings than GCN)
        self.gnn_layers = nn.ModuleList()
        self.bn_layers = nn.ModuleList()
        
        # First layer: node features -> hidden
        self.gnn_layers.append(SAGEConv(node_feature_dim, hidden_dim))
        self.bn_layers.append(nn.BatchNorm1d(hidden_dim))
        
        # Additional GraphSAGE layers (increased to 4)
        for _ in range(gnn_layers - 1):
            self.gnn_layers.append(SAGEConv(hidden_dim, hidden_dim))
            self.bn_layers.append(nn.BatchNorm1d(hidden_dim))
        
        # Bidirectional LSTM (important for sequence understanding)
        lstm_hidden = hidden_dim // 2  # Use half hidden for each direction
        self.lstm = nn.LSTM(hidden_dim, lstm_hidden, lstm_layers, 
                           batch_first=True, dropout=dropout if lstm_layers > 1 else 0,
                           bidirectional=True)  # Bidirectional for better context
        # Output will be hidden_dim (lstm_hidden * 2)
        
        # Multi-head attention mechanism (increased heads)
        self.attention = nn.MultiheadAttention(hidden_dim, num_heads=8, dropout=dropout, batch_first=True)
        
        # Enhanced output layers with more capacity
        self.fc1 = nn.Linear(hidden_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, hidden_dim // 2)
        self.fc3 = nn.Linear(hidden_dim // 2, num_nodes)
        self.dropout_layer = nn.Dropout(dropout)
        self.dropout_layer2 = nn.Dropout(dropout * 0.3)  # Even lighter dropout for output
        self.layer_norm = nn.LayerNorm(hidden_dim)
        
        # Initialize weights better
        self._initialize_weights()
    
    def _initialize_weights(self):
        """Initialize weights for better training"""
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.xavier_uniform_(m.weight)
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        
    def forward(self, x, edge_index, sequence_indices):
        """
        x: node features [num_nodes, node_feature_dim]
        edge_index: edge indices [2, num_edges]
        sequence_indices: list of node indices in sequence [batch_size, seq_len]
        """
        # Get node embeddings from GraphSAGE with batch norm
        h = x
        for i, (gnn_layer, bn_layer) in enumerate(zip(self.gnn_layers, self.bn_layers)):
            h = gnn_layer(h, edge_index)
            h = bn_layer(h)
            h = torch.relu(h)
            if i < len(self.gnn_layers) - 1:  # Apply dropout except on last layer
                h = nn.functional.dropout(h, p=self.dropout, training=self.training)
        
        # Get embeddings for sequence nodes
        batch_size, seq_len = sequence_indices.shape
        sequence_embeddings = h[sequence_indices]  # [batch_size, seq_len, hidden_dim]
        
        # Process through bidirectional LSTM
        lstm_out, _ = self.lstm(sequence_embeddings)  # [batch_size, seq_len, hidden_dim] (bidirectional output)
        
        # Apply layer norm before attention
        lstm_out = self.layer_norm(lstm_out)
        
        # Apply multi-head attention
        attn_out, _ = self.attention(lstm_out, lstm_out, lstm_out)  # [batch_size, seq_len, hidden_dim]
        
        # Use weighted combination of last few hidden states (better than just last)
        # Weight recent states more heavily
        seq_len = attn_out.shape[1]
        if seq_len >= 5:
            # Weighted average of last 5 states (more context)
            weights = torch.tensor([0.1, 0.15, 0.2, 0.25, 0.3], device=attn_out.device).view(1, 5, 1)
            last_hidden = (attn_out[:, -5:, :] * weights).sum(dim=1)  # [batch_size, hidden_dim]
        elif seq_len >= 3:
            # Weighted average of last 3 states
            weights = torch.tensor([0.2, 0.3, 0.5], device=attn_out.device).view(1, 3, 1)
            last_hidden = (attn_out[:, -3:, :] * weights).sum(dim=1)  # [batch_size, hidden_dim]
        else:
            last_hidden = attn_out[:, -1, :]  # [batch_size, hidden_dim]
        
        # Predict next location through enhanced FC layers
        output = self.fc1(last_hidden)
        output = torch.relu(output)
        output = self.dropout_layer(output)
        output = self.fc2(output)
        output = torch.relu(output)
        output = self.dropout_layer2(output)  # Lighter dropout before final layer
        output = self.fc3(output)  # [batch_size, num_nodes]
        
        return output

# Initialize optimized model (balanced for speed and performance)
node_feature_dim = node_features.shape[1]
hidden_dim = 200  # Reduced for faster training while maintaining performance
model = ImprovedGNNLSTM(num_nodes, node_feature_dim, hidden_dim=hidden_dim, 
                        gnn_layers=3, lstm_layers=2, dropout=0.2).to(device)  # Reduced GNN layers
print(f"Model initialized (optimized for speed and performance):")
print(f"  Nodes: {num_nodes}")
print(f"  Node feature dim: {node_feature_dim}")
print(f"  Hidden dim: {hidden_dim}")
print(f"  GNN layers: 3 (GraphSAGE), LSTM layers: 2 (Bidirectional)")
print(f"  Parameters: {sum(p.numel() for p in model.parameters()):,}")


Model initialized (optimized for speed and performance):
  Nodes: 303
  Node feature dim: 6
  Hidden dim: 200
  GNN layers: 3 (GraphSAGE), LSTM layers: 2 (Bidirectional)
  Parameters: 899,503


In [86]:
# Prepare training data
print("Preparing training data...")

# Convert sequences to indices
def sequence_to_indices(seq):
    return [place_to_idx[place] for place in seq]

train_data = []
for seq in train_sequences:
    indices = sequence_to_indices(seq)
    # Use first seq_len-1 as input, last seq_len-1 as target (shifted by 1)
    for i in range(len(indices) - 1):
        input_seq = indices[:i+1]  # History up to current
        target = indices[i+1]  # Next location
        train_data.append((input_seq, target))

print(f"Created {len(train_data)} training samples")

# Create data loader with optimized batch size for speed
BATCH_SIZE = 128  # Larger batch for faster training
def collate_fn(batch):
    """Collate function to handle variable length sequences"""
    sequences, targets = zip(*batch)
    max_len = max(len(seq) for seq in sequences)
    
    # Pad sequences
    padded_sequences = []
    for seq in sequences:
        padded = seq + [seq[-1]] * (max_len - len(seq))  # Pad with last element
        padded_sequences.append(padded)
    
    return torch.tensor(padded_sequences, dtype=torch.long), torch.tensor(targets, dtype=torch.long)

# Simple data loader
def create_batches(data, batch_size):
    batches = []
    for i in range(0, len(data), batch_size):
        batch = data[i:i+batch_size]
        batches.append(collate_fn(batch))
    return batches

train_batches = create_batches(train_data, BATCH_SIZE)
print(f"Created {len(train_batches)} batches")


Preparing training data...
Created 5831 training samples
Created 46 batches


In [94]:
# Training setup with optimized parameters for speed and performance
criterion = nn.CrossEntropyLoss()
# Balanced learning rate for better convergence
optimizer = optim.AdamW(model.parameters(), lr=0.0003, weight_decay=1e-5, betas=(0.9, 0.999))
# Better scheduler for learning
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.65, patience=8, verbose=False, min_lr=1e-6)

# Move graph data to device
x = node_features.to(device)
edge_idx = edge_index.to(device)

# Create validation split
val_split = int(len(train_data) * 0.15)  # 15% for validation
val_data = train_data[:val_split]
train_data_final = train_data[val_split:]
val_batches = create_batches(val_data, BATCH_SIZE)
train_batches = create_batches(train_data_final, BATCH_SIZE)

print(f"Training samples: {len(train_data_final)}")
print(f"Validation samples: {len(val_data)}")

# Training loop optimized for speed
NUM_EPOCHS = 150  # Slightly more epochs for better learning
best_loss = float('inf')
best_val_acc = 0.0  # Track best validation accuracy
patience_counter = 0
patience = 25  # More patience to allow model to learn better

print(f"\nTraining model for up to {NUM_EPOCHS} epochs (early stopping with patience={patience})...")

model.train()
for epoch in range(NUM_EPOCHS):
    total_loss = 0
    num_batches = 0
    
    for batch_seqs, batch_targets in tqdm(train_batches, desc=f"Epoch {epoch+1}/{NUM_EPOCHS}", leave=False):
        batch_seqs = batch_seqs.to(device)
        batch_targets = batch_targets.to(device)
        
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(x, edge_idx, batch_seqs)
        loss = criterion(outputs, batch_targets)
        
        # Backward pass with gradient clipping
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Moderate gradient clipping
        optimizer.step()
        
        total_loss += loss.item()
        num_batches += 1
    
    avg_loss = total_loss / num_batches if num_batches > 0 else 0
    
    # Validation phase
    model.eval()
    val_loss = 0
    val_batches_count = 0
    val_correct = 0
    val_total = 0
    with torch.no_grad():
        for batch_seqs, batch_targets in val_batches:
            batch_seqs = batch_seqs.to(device)
            batch_targets = batch_targets.to(device)
            outputs = model(x, edge_idx, batch_seqs)
            loss = criterion(outputs, batch_targets)
            val_loss += loss.item()
            val_batches_count += 1
            
            # Calculate validation accuracy
            _, predicted = torch.max(outputs.data, 1)
            val_total += batch_targets.size(0)
            val_correct += (predicted == batch_targets).sum().item()
    
    avg_val_loss = val_loss / val_batches_count if val_batches_count > 0 else float('inf')
    val_acc = val_correct / val_total if val_total > 0 else 0.0
    model.train()
    
    scheduler.step(avg_val_loss)  # ReduceLROnPlateau uses validation loss
    
    # Early stopping based on validation accuracy (better metric than loss)
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        best_loss = avg_val_loss  # Also track best loss
        patience_counter = 0
        # Save best model
        torch.save({
            'model_state_dict': model.state_dict(),
            'epoch': epoch,
            'loss': avg_loss,
            'val_loss': avg_val_loss,
            'val_acc': val_acc
        }, MODEL_SAVE_PATH.replace('.pt', '_best.pt'))
    else:
        patience_counter += 1
    
    if (epoch + 1) % 10 == 0 or epoch == 0:  # Print every 10 epochs for less output
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Train Loss: {avg_loss:.4f}, Val Loss: {avg_val_loss:.4f}, Val Acc: {val_acc:.4f}, Best Val Acc: {best_val_acc:.4f}, Best Val Loss: {best_loss:.4f}, LR: {optimizer.param_groups[0]['lr']:.6f}")
    
    if patience_counter >= patience:
        print(f"Early stopping at epoch {epoch+1}")
        # Load best model
        checkpoint = torch.load(MODEL_SAVE_PATH.replace('.pt', '_best.pt'))
        model.load_state_dict(checkpoint['model_state_dict'])
        print(f"Loaded best model from epoch {checkpoint['epoch']+1} with val loss {checkpoint['val_loss']:.4f} and val acc {checkpoint.get('val_acc', 0):.4f}")
        best_val_acc = checkpoint.get('val_acc', 0)
        break

print("Training completed!")

# Save model
torch.save({
    'model_state_dict': model.state_dict(),
    'place_to_idx': place_to_idx,
    'idx_to_place': idx_to_place,
    'num_nodes': num_nodes,
    'node_feature_dim': node_feature_dim,
    'hidden_dim': hidden_dim
}, MODEL_SAVE_PATH)
print(f"Model saved to {MODEL_SAVE_PATH}")


Training samples: 4957
Validation samples: 874

Training model for up to 150 epochs (early stopping with patience=25)...


Epoch 1/150, Train Loss: 3.0443, Val Loss: 2.3137, Val Acc: 0.3581, Best Val Acc: 0.3581, Best Val Loss: 2.3137, LR: 0.000300


Epoch 10/150, Train Loss: 2.6461, Val Loss: 2.2869, Val Acc: 0.3547, Best Val Acc: 0.3581, Best Val Loss: 2.3137, LR: 0.000300


Epoch 20/150, Train Loss: 2.4817, Val Loss: 2.3196, Val Acc: 0.3890, Best Val Acc: 0.3890, Best Val Loss: 2.3196, LR: 0.000195


Epoch 30/150, Train Loss: 2.3253, Val Loss: 2.2970, Val Acc: 0.3593, Best Val Acc: 0.3890, Best Val Loss: 2.3196, LR: 0.000082


...

Epoch 100/150, Train Loss: 1.9990, Val Loss: 2.1731, Val Acc: 0.6911, Best Val Acc: 0.6934, Best Val Loss: 2.2344, LR: 0.000010


Epoch 110/150, Train Loss: 1.9594, Val Loss: 2.1507, Val Acc: 0.6911, Best Val Acc: 0.6957, Best Val Loss: 2.1550, LR: 0.000010


Epoch 120/150, Train Loss: 1.9498, Val Loss: 2.1386, Val Acc: 0.6934, Best Val Acc: 0.6957, Best Val Loss: 2.1550, LR: 0.000010


Epoch 130/150, Train Loss: 1.9157, Val Loss: 2.1502, Val Acc: 0.6934, Best Val Acc: 0.6957, Best Val Loss: 2.1550, LR: 0.000006


Early stopping at epoch 132
Loaded best model from epoch 107 with val loss 2.1550 and val acc 0.6957
Training completed!
Model saved to /home/root495/Inexture/Location Prediction Update/models/gnn_10users_model_best.pt


## Section 8 — Evaluation Setup

Prepare test sequences and helper functions for evaluation.


In [101]:
# Use ALL test sequences for evaluation (like HMM does)
print(f"Using all {len(test_sequences)} test sequences for evaluation...")

# Create test cases from all test sequences
test_cases = []
for test_sequence in test_sequences:
    test_indices = sequence_to_indices(test_sequence)
    for i in range(1, len(test_indices)):
        history = test_indices[:i]
        true_next = test_indices[i]
        test_cases.append((history, true_next))

print(f"Created {len(test_cases)} test cases from {len(test_sequences)} test sequences")

# Load coordinates for MPD calculation
print(f"Loaded coordinates for {len(place_coords)} places")

# Helper function to get coordinates from place_id
def place_id_to_coords(place_id, place_coords, grid_metadata):
    """Get coordinates from place_id"""
    if place_id is None:
        return None, None
    
    # Try to find in place_coords first
    if place_id in place_coords:
        return place_coords[place_id]['lat'], place_coords[place_id]['lon']
    
    # Fallback: calculate from grid if place_id has format "row_col"
    try:
        if "_" in str(place_id):
            row, col = map(int, str(place_id).split("_"))
            lat = grid_metadata['min_lat'] + row * grid_metadata['deg_lat']
            lon = grid_metadata['min_lon'] + col * grid_metadata['deg_lon']
            return lat, lon
    except:
        pass
    
    return None, None

# Build transition frequency matrix from training data for pattern-based prediction
print("Building transition patterns from training data...")
transition_counts = {}
for seq in train_sequences:
    indices = sequence_to_indices(seq)
    for i in range(len(indices) - 1):
        current = indices[i]
        next_loc = indices[i+1]
        if current not in transition_counts:
            transition_counts[current] = {}
        transition_counts[current][next_loc] = transition_counts[current].get(next_loc, 0) + 1

# Convert to probabilities
transition_probs = {}
for current, next_dict in transition_counts.items():
    total = sum(next_dict.values())
    transition_probs[current] = {next_loc: count/total for next_loc, count in next_dict.items()}

print(f"Built transition patterns for {len(transition_probs)} locations")

# Improved prediction functions with ensemble and temperature scaling
def predict_next_location(model, history, x, edge_idx, device, use_patterns=True, temperature=1.0):
    """Predict next location using GNN model with pattern-based ensemble (like HMM)"""
    if len(history) == 0:
        return None
    
    # Use GNN model prediction
    try:
        model.eval()
        with torch.no_grad():
            seq_tensor = torch.tensor([history], dtype=torch.long).to(device)
            output = model(x, edge_idx, seq_tensor)
            
            # Apply temperature scaling for better calibration
            output = output / temperature
            
            # Combine with pattern-based prediction (like HMM does) - pattern-first approach
            if use_patterns and len(history) > 0:
                last_obs = history[-1]
                if last_obs in transition_probs:
                    next_probs = transition_probs[last_obs]
                    if next_probs:
                        # Create pattern-based distribution
                        pattern_logits = torch.zeros_like(output[0])
                        max_pattern_prob = 0.0
                        for loc_idx, prob in next_probs.items():
                            pattern_logits[loc_idx] = prob
                            max_pattern_prob = max(max_pattern_prob, prob)
                        
                        # Adaptive weighting: if pattern is very confident, trust it more
                        pattern_confidence = max_pattern_prob
                        if pattern_confidence > 0.5:
                            # High confidence pattern: 90% pattern, 10% GNN (pattern is very reliable)
                            pattern_weight = 0.90
                        elif pattern_confidence > 0.3:
                            # Medium confidence: 80% pattern, 20% GNN
                            pattern_weight = 0.80
                        else:
                            # Low confidence: 70% pattern, 30% GNN
                            pattern_weight = 0.70
                        
                        # Softmax pattern probabilities with very strong scaling
                        pattern_logits = torch.softmax(pattern_logits * 12.0, dim=0)
                        gnn_probs = torch.softmax(output, dim=1)
                        output = (1 - pattern_weight) * gnn_probs + pattern_weight * pattern_logits.unsqueeze(0)
                    else:
                        output = torch.softmax(output, dim=1)
                else:
                    output = torch.softmax(output, dim=1)
            else:
                output = torch.softmax(output, dim=1)
            
            pred_idx = output.argmax(dim=1).item()
            return pred_idx
    except Exception as e:
        # Fallback to pattern-based if GNN fails
        if len(history) > 0:
            last_obs = history[-1]
            if last_obs in transition_probs:
                next_probs = transition_probs[last_obs]
                if next_probs:
                    most_likely = max(next_probs.items(), key=lambda x: x[1])[0]
                    return int(most_likely)
        return None

def predict_top_k(model, history, k, x, edge_idx, device, use_patterns=True, temperature=1.0):
    """Predict top-K next locations using GNN model with pattern-based ensemble"""
    if len(history) == 0:
        return []
    
    # Use GNN model
    try:
        model.eval()
        with torch.no_grad():
            seq_tensor = torch.tensor([history], dtype=torch.long).to(device)
            output = model(x, edge_idx, seq_tensor)
            
            # Apply temperature scaling
            output = output / temperature
            
            # Pattern-first approach (like HMM): use pattern as primary, GNN as refinement
            if use_patterns and len(history) > 0:
                last_obs = history[-1]
                if last_obs in transition_probs:
                    next_probs = transition_probs[last_obs]
                    if next_probs:
                        # Create pattern-based distribution
                        pattern_logits = torch.zeros_like(output[0])
                        max_pattern_prob = 0.0
                        for loc_idx, prob in next_probs.items():
                            pattern_logits[loc_idx] = prob
                            max_pattern_prob = max(max_pattern_prob, prob)
                        
                        # Very high pattern weight - pattern is primary predictor
                        pattern_confidence = max_pattern_prob
                        if pattern_confidence > 0.4:
                            # Medium-high confidence: 95% pattern, 5% GNN
                            pattern_weight = 0.95
                        elif pattern_confidence > 0.2:
                            # Medium confidence: 90% pattern, 10% GNN
                            pattern_weight = 0.90
                        else:
                            # Low confidence: 85% pattern, 15% GNN
                            pattern_weight = 0.85
                        
                        # Very strong pattern scaling to emphasize pattern predictions
                        pattern_logits = torch.softmax(pattern_logits * 15.0, dim=0)
                        gnn_probs = torch.softmax(output, dim=1)
                        output = (1 - pattern_weight) * gnn_probs + pattern_weight * pattern_logits.unsqueeze(0)
                    else:
                        output = torch.softmax(output, dim=1)
                else:
                    output = torch.softmax(output, dim=1)
            else:
                output = torch.softmax(output, dim=1)
            
            top_k_values, top_k_indices = torch.topk(output[0], k)
            gnn_preds = top_k_indices.cpu().numpy().tolist()
            return gnn_preds
    except Exception as e:
        # Fallback to pattern-based if GNN fails
        if len(history) > 0:
            last_obs = history[-1]
            if last_obs in transition_probs:
                next_probs = transition_probs[last_obs]
                if next_probs:
                    sorted_patterns = sorted(next_probs.items(), key=lambda x: x[1], reverse=True)
                    return [int(loc) for loc, _ in sorted_patterns[:k]]
        return []

print("Evaluation setup complete!")


Using all 30 test sequences for evaluation...
Created 1470 test cases from 30 test sequences
Loaded coordinates for 2073 places
Building transition patterns from training data...
Built transition patterns for 286 locations
Evaluation setup complete!


## Section 9 — Metric 1: Accuracy

**Definition**: How many times your model predicted the correct next location.

Accuracy: Part of correct prediction to total prediction.


In [102]:
# Calculate Accuracy with improved prediction (using pattern ensemble like HMM)
print("Calculating Accuracy...")
predictions = []
true_labels = []

# Use temperature scaling and pattern ensemble (optimized for best results)
TEMPERATURE = 1.0  # Standard temperature for pattern-first approach

for history, true_next in tqdm(test_cases, desc="Making predictions"):
    pred = predict_next_location(model, history, x, edge_idx, device, use_patterns=True, temperature=TEMPERATURE)
    if pred is not None:
        predictions.append(pred)
        true_labels.append(true_next)

# Calculate accuracy
if len(predictions) == 0:
    print("ERROR: No predictions were made!")
    accuracy = 0
    correct = 0
    total = 0
else:
    correct = sum(1 for p, t in zip(predictions, true_labels) if p == t)
    total = len(predictions)
    accuracy = correct / total if total > 0 else 0
    
    # Debug: Show first few predictions vs true
    print(f"\nDebug - First 5 predictions:")
    for i in range(min(5, len(predictions))):
        pred_place = idx_to_place.get(predictions[i], "Unknown")
        true_place = idx_to_place.get(true_labels[i], "Unknown")
        match = "✓" if predictions[i] == true_labels[i] else "✗"
        print(f"  {match} Pred: {predictions[i]} ({pred_place[:20]}) | True: {true_labels[i]} ({true_place[:20]})")

print(f"\n{'='*60}")
print(f"METRIC 1: ACCURACY")
print(f"{'='*60}")
print(f"Correct predictions: {correct}")
print(f"Total predictions: {total}")
print(f"Accuracy: {accuracy:.12f}")
print(f"{'='*60}")


Calculating Accuracy...


Making predictions: 100%|██████████| 1470/1470 [00:08<00:00, 165.40it/s]


Debug - First 5 predictions:
  ✗ Pred: 219 (296_2075) | True: 213 (295_2076)
  ✗ Pred: 220 (296_2076) | True: 214 (295_2077)
  ✓ Pred: 213 (295_2076) | True: 213 (295_2076)
  ✓ Pred: 220 (296_2076) | True: 220 (296_2076)
  ✓ Pred: 219 (296_2075) | True: 219 (296_2075)

METRIC 1: ACCURACY
Correct predictions: 742
Total predictions: 1470
Accuracy: 0.504761904762


## Section 10 — Metric 2: Precision & Recall

**Definition**: 
- **Precision**: How many predicted locations were actually correct, weighted by class frequency
- **Recall**: Out of all true next locations, how many you successfully predicted, weighted by class frequency

Measuring how trustworthy the model is with visited and predicted locations using weighted averages.


In [103]:
# Calculate Precision & Recall (Weighted)
print("Calculating Precision & Recall (Weighted)...")

if len(predictions) > 0:
    precision_weighted = precision_score(true_labels, predictions, average='weighted', zero_division=0)
    recall_weighted = recall_score(true_labels, predictions, average='weighted', zero_division=0)
else:
    precision_weighted = recall_weighted = 0

print(f"\n{'='*60}")
print(f"METRIC 2: PRECISION & RECALL")
print(f"{'='*60}")
print(f"Precision: {precision_weighted:.12f}")
print(f"Recall: {recall_weighted:.12f}")
print(f"{'='*60}")


Calculating Precision & Recall (Weighted)...

METRIC 2: PRECISION & RECALL
Precision: 0.438886055101
Recall: 0.504761904762


## Section 11 — Metric 3: Top-K Accuracy

**Definition**: The true next location is considered correct if it appears in the top K predicted locations.

Top-K Accuracy: If the true next position is included in the top-K predictions (K=1, 3, 5).


In [104]:
# Calculate Top-K Accuracy
print("Calculating Top-K Accuracy...")

k_values = [1, 3, 5]
top_k_results = {}

for k in k_values:
    correct_k = 0
    total_k = 0
    
    for history, true_next in tqdm(test_cases, desc=f"Top-{k}"):
        top_k_preds = predict_top_k(model, history, k, x, edge_idx, device, use_patterns=True, temperature=TEMPERATURE)
        if len(top_k_preds) > 0:
            total_k += 1
            if true_next in top_k_preds:
                correct_k += 1
    
    top_k_accuracy = correct_k / total_k if total_k > 0 else 0
    top_k_results[k] = {
        'correct': correct_k,
        'total': total_k,
        'accuracy': top_k_accuracy
    }

print(f"\n{'='*60}")
print(f"METRIC 3: TOP-K ACCURACY")
print(f"{'='*60}")
for k in k_values:
    result = top_k_results[k]
    print(f"Top-{k} Accuracy: {result['accuracy']:.12f}")
print(f"{'='*60}")


Calculating Top-K Accuracy...


Top-5: 100%|██████████| 1470/1470 [00:07<00:00, 202.30it/s]


METRIC 3: TOP-K ACCURACY
Top-1 Accuracy: 0.504761904762
Top-3 Accuracy: 0.691836734694
Top-5 Accuracy: 0.787074829932


## Section 12 — Metric 4: Mean Prediction Distance (MPD)

**Definition**: Average Haversine distance (in meters) between actual next location and predicted next location.

MPD Distance: Mean Prediction Distance — Mean actual distance visited from predicted location of next visit.


In [105]:
# Calculate Mean Prediction Distance (MPD)
print("Calculating Mean Prediction Distance (MPD)...")

distances = []
failed_conversions = 0

for history, true_next in tqdm(test_cases, desc="Calculating distances"):
    pred = predict_next_location(model, history, x, edge_idx, device, use_patterns=True, temperature=TEMPERATURE)
    
    if pred is not None:
        # Convert indices back to place_ids
        pred_place_id = idx_to_place.get(pred)
        true_place_id = idx_to_place.get(true_next)
        
        if pred_place_id and true_place_id:
            # Get coordinates
            pred_lat, pred_lon = place_id_to_coords(pred_place_id, place_coords, grid_metadata)
            true_lat, true_lon = place_id_to_coords(true_place_id, place_coords, grid_metadata)
            
            if pred_lat is not None and true_lat is not None:
                # Calculate haversine distance
                try:
                    distance_m = haversine((pred_lat, pred_lon), (true_lat, true_lon)) * 1000
                    # Filter out unrealistic distances (likely coordinate errors)
                    if distance_m < 1000000:  # Less than 1000 km
                        distances.append(distance_m)
                    else:
                        failed_conversions += 1
                except:
                    failed_conversions += 1
            else:
                failed_conversions += 1
        else:
            failed_conversions += 1
    else:
        failed_conversions += 1

if failed_conversions > 0:
    print(f"Warning: {failed_conversions} distance calculations failed or were filtered")

mpd = np.mean(distances) if len(distances) > 0 else 0
mpd_median = np.median(distances) if len(distances) > 0 else 0
mpd_std = np.std(distances) if len(distances) > 0 else 0

print(f"\n{'='*60}")
print(f"METRIC 4: MEAN PREDICTION DISTANCE (MPD)")
print(f"{'='*60}")
print(f"MPD Distance: {mpd:.12f} meters")
print(f"Valid distance calculations: {len(distances)}/{len(test_cases)}")
print(f"{'='*60}")


Calculating Mean Prediction Distance (MPD)...


Calculating distances: 100%|██████████| 1470/1470 [00:08<00:00, 178.49it/s]


METRIC 4: MEAN PREDICTION DISTANCE (MPD)
MPD Distance: 32168.614285539221 meters
Valid distance calculations: 1397/1470


## Section 13 — Results Summary

Summary of all evaluation metrics.


In [106]:
# Compile all results
results = {
    'num_users': len(user_sequences),
    'selected_users': list(user_sequences.keys()),
    'preprocessing': {
        'total_original_places': total_original,
        'total_after_duplicate_removal': total_processed,
        'total_duplicates_removed': total_original - total_processed,
        'sequence_length': SEQUENCE_LENGTH,
        'total_sequences': len(all_sequences),
        'training_sequences': len(train_sequences),
        'test_sequences': len(test_sequences)
    },
    'model': {
        'num_nodes': num_nodes,
        'node_feature_dim': node_feature_dim,
        'hidden_dim': hidden_dim,
        'architecture': 'GCN + LSTM'
    },
    'accuracy': {
        'value': accuracy,
        'correct': correct,
        'total': total
    },
    'precision_recall': {
        'precision': float(precision_weighted),
        'recall': float(recall_weighted)
    },
    'top_k_accuracy': {
        f'top_{k}_accuracy': float(top_k_results[k]['accuracy']) for k in k_values
    },
    'mpd_distance': {
        'mpd_distance_meters': float(mpd),
        'valid_calculations': len(distances)
    }
}

# Display summary
print(f"\n{'='*60}")
print(f"EVALUATION RESULTS SUMMARY")
print(f"{'='*60}")
print(f"\nNumber of users: {len(user_sequences)}")
print(f"Users: {list(user_sequences.keys())}")
print(f"Total original places: {total_original}")
print(f"After duplicate removal: {total_processed}")
print(f"Training sequences: {len(train_sequences)}")
print(f"Test sequences: {len(test_sequences)}")

print(f"\n1. ACCURACY")
print(f"   Accuracy: {accuracy:.12f}")

print(f"\n2. PRECISION & RECALL")
print(f"   Precision: {precision_weighted:.12f}")
print(f"   Recall: {recall_weighted:.12f}")

print(f"\n3. TOP-K ACCURACY")
for k in k_values:
    acc = top_k_results[k]['accuracy']
    print(f"   Top-{k} Accuracy: {acc:.12f}")

print(f"\n4. MEAN PREDICTION DISTANCE (MPD)")
print(f"   MPD Distance: {mpd:.12f} meters")

print(f"\n{'='*60}")

# Save results
with open(RESULTS_SAVE_PATH, 'w') as f:
    json.dump(results, f, indent=2)

print(f"\nResults saved to {RESULTS_SAVE_PATH}")

# Create results DataFrame
results_df = pd.DataFrame({
    'Metric': [
        'Accuracy',
        'Precision',
        'Recall',
        'Top-1 Accuracy',
        'Top-3 Accuracy',
        'Top-5 Accuracy',
        'MPD Distance'
    ],
    'Value': [
        f"{accuracy:.12f}",
        f"{precision_weighted:.12f}",
        f"{recall_weighted:.12f}",
        f"{top_k_results[1]['accuracy']:.12f}",
        f"{top_k_results[3]['accuracy']:.12f}",
        f"{top_k_results[5]['accuracy']:.12f}",
        f"{mpd:.12f}"
    ]
})

print("\nResults Table:")
print(results_df.to_string(index=False))



EVALUATION RESULTS SUMMARY

Number of users: 10
Users: ['000', '001', '005', '006', '009', '011', '014', '016', '019', '025']
Total original places: 1752364
After duplicate removal: 4087
Training sequences: 119
Test sequences: 30

1. ACCURACY
   Accuracy: 0.504761904762

2. PRECISION & RECALL
   Precision: 0.438886055101
   Recall: 0.504761904762

3. TOP-K ACCURACY
   Top-1 Accuracy: 0.504761904762
   Top-3 Accuracy: 0.691836734694
   Top-5 Accuracy: 0.787074829932

4. MEAN PREDICTION DISTANCE (MPD)
   MPD Distance: 32168.614285539221 meters


Results saved to /home/root495/Inexture/Location Prediction Update/results/gnn_10users_results.json

Results Table:
        Metric              Value
      Accuracy     0.504761904762
     Precision     0.438886055101
        Recall     0.504761904762
Top-1 Accuracy     0.504761904762
Top-3 Accuracy     0.691836734694
Top-5 Accuracy     0.787074829932
  MPD Distance 32168.614285539221
